## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-17-20-53-09 +0000,nyt,UK Rejoins EU’s Erasmus Student Exchange Progr...,https://www.nytimes.com/2025/12/17/world/europ...
1,2025-12-17-20-49-38 +0000,nyt,"Norman Podhoretz, Literary Lion of Neoconserva...",https://www.nytimes.com/2025/12/16/us/politics...
2,2025-12-17-20-48-34 +0000,wapo,"To make newborn investment accounts a reality,...",https://www.washingtonpost.com
3,2025-12-17-20-48-00 +0000,wsj,The Senate passed the sweeping defense-policy ...,https://www.wsj.com/politics/national-security...
4,2025-12-17-20-45-22 +0000,bbc,Heavy rains worsen conditions of Gaza's displa...,https://www.bbc.com/news/articles/cx2e6r10rdeo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,48
115,reiner,25
270,bondi,22
303,new,19
100,venezuela,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
320,2025-12-17-01-32-56 +0000,nypost,Trump orders blockade of oil tankers from ‘for...,https://nypost.com/2025/12/16/us-news/trump-or...,130
28,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...,117
342,2025-12-17-00-10-52 +0000,cbc,Trump orders blockade of 'sanctioned oil tanke...,https://www.cbc.ca/news/world/trump-venezuela-...,111
105,2025-12-17-17-47-56 +0000,bbc,"What we know about Nick Reiner, son charged wi...",https://www.bbc.com/news/articles/cvgj14l3v8vo...,107
52,2025-12-17-19-13-00 +0000,wsj,Major U.S. Stock Indexes Fall; Oil Prices Gain...,https://www.wsj.com/finance/stocks/u-s-futures...,106


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
320,130,2025-12-17-01-32-56 +0000,nypost,Trump orders blockade of oil tankers from ‘for...,https://nypost.com/2025/12/16/us-news/trump-or...
105,107,2025-12-17-17-47-56 +0000,bbc,"What we know about Nick Reiner, son charged wi...",https://www.bbc.com/news/articles/cvgj14l3v8vo...
34,76,2025-12-17-19-55-55 +0000,nypost,Top cop defends Bondi Beach officer accused of...,https://nypost.com/2025/12/17/world-news/top-c...
198,72,2025-12-17-12-59-17 +0000,nypost,Warner Bros. Discovery board urges shareholder...,https://nypost.com/2025/12/17/business/warner-...
28,69,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...
338,54,2025-12-17-00-24-00 +0000,wsj,New Video Shows Suspected Brown Gunman’s Movem...,https://www.wsj.com/us-news/brown-university-s...
10,51,2025-12-17-20-36-00 +0000,wsj,FCC Chair Brendan Carr told a Senate oversight...,https://www.wsj.com/politics/policy/fcc-isnt-a...
317,44,2025-12-17-02-00-00 +0000,wsj,Tuesday’s jobs report was the latest ominous s...,https://www.wsj.com/economy/jobs/white-collar-...
119,42,2025-12-17-17-26-37 +0000,nypost,"Nick Reiner wears only vacant stare, suicide s...",https://nypost.com/2025/12/17/us-news/nick-rei...
349,41,2025-12-16-23-59-00 +0000,wsj,The Trump administration expanded its travel b...,https://www.wsj.com/politics/policy/trump-admi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
